# GISTDA Wildfire Machine Learning Training

## Import Library Packages

#### Core Libraries
- **numpy**: A powerful library for numerical computing, providing support for arrays, matrices, and a wide range of mathematical operations.
- **pandas**: A versatile library for data manipulation and analysis, offering easy-to-use data structures like `DataFrame` for handling tabular data.

#### Visualization Libraries
- **matplotlib.pyplot**: A plotting library for creating static, animated, and interactive visualizations in Python.
- **seaborn**: Built on top of Matplotlib, this library provides a high-level interface for drawing attractive and informative statistical graphics.

#### Geospatial Libraries
- **rasterio**: A library for reading and writing geospatial raster data, commonly used for remote sensing and GIS applications.
  - **rasterio.windows.Window**: A tool to define a specific rectangular region (window) within a raster dataset for operations.

#### Logging
- **logging**: A built-in Python module to record log messages, useful for debugging and monitoring program execution.

#### File Handling
- **os**: A module providing tools to interact with the operating system, such as reading/writing files and directory management.
- **pickle**: A module for serializing and deserializing Python objects for storage or transmission.

#### Data Preprocessing
- **sklearn.preprocessing.MinMaxScaler**: A preprocessing tool from scikit-learn that scales and normalizes data within a specified range, often [0, 1].

#### Display Utilities
- **IPython.display.display**: A utility to control the display of outputs in Jupyter notebooks.
- **IPython.display.Markdown**: Enables the rendering of Markdown text in a Jupyter notebook.

#### Deep Learning Libraries
- **tensorflow**: A popular open-source framework for machine learning and deep learning. It supports building and training neural networks efficiently.
  - **tensorflow.keras.models.Sequential**: A simple model-building API for stacking layers sequentially.
  - **tensorflow.keras.layers.Dense**: A fully connected neural network layer.
  - **tensorflow.keras.layers.Dropout**: A regularization technique to prevent overfitting by randomly dropping neurons during training.
  - **tensorflow.keras.callbacks.EarlyStopping**: Stops training when a monitored metric stops improving.
  - **tensorflow.keras.callbacks.ReduceLROnPlateau**: Adjusts the learning rate when a metric stops improving.

#### Evaluation Metrics
- **sklearn.metrics.classification_report**: Generates a detailed report of classification metrics, including precision, recall, and F1 score.
- **sklearn.metrics.confusion_matrix**: Computes a confusion matrix to evaluate classification accuracy.
- **sklearn.metrics.roc_curve**: Computes Receiver Operating Characteristic (ROC) curve metrics.
- **sklearn.metrics.auc**: Computes the Area Under the Curve (AUC) for a ROC curve.

#### Typing
- **typing**: Provides tools for type annotations to improve code clarity and static analysis.
  - **Tuple, Dict, Any, List**: Common data types used in type hinting.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
import logging
import os
import pickle
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, Markdown
from rasterio.windows import Window
from typing import Tuple, Dict, Any, List
from dask import delayed, compute

import tensorflow as tf
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import classification_report, confusion_matrix

## Import Raster GeoTIFF files

This code reads large raster files in manageable chunks, processes each chunk, and stores the data in a consolidated DataFrame.

1. **Folder Path and Chunk Size**: Sets the path to the folder containing raster files (`raster_train_file_path`) and defines the chunk size (`CHUNK_SIZE`).

2. **`read_raster_in_chunks` Function**:
   - Reads a raster file chunk-by-chunk.
   - For each chunk, it extracts data from all bands, along with the pixel coordinates.
   - Stores each chunk's data in a temporary DataFrame, which is added to a list.

3. **Task Creation**:
   - Creates a list of tasks using Dask's `@delayed` to process each raster file in parallel.

4. **Compute and Combine**:
   - Computes each task to get individual DataFrames for each file, then combines them into a final DataFrame (`final_df`).

5. **Debug Outputs**:
   - Prints the shape, columns, and a sample of the final DataFrame for inspection.

In [ ]:
# Define the folder containing the raster files
raster_train_file_path = r'Raster_Train'

# Parameters for chunk size
CHUNK_SIZE = 1024

@delayed
def read_raster_in_chunks(raster_path, file, root):
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        num_bands = src.count
        band_names = [f'B{str(i).zfill(2)}' for i in range(1, num_bands + 1)]
        chunk_dfs = []
        
        # Loop over the raster in chunks
        for row in range(0, height, CHUNK_SIZE):
            for col in range(0, width, CHUNK_SIZE):
                window = Window(col_off=col, row_off=row, 
                              width=min(CHUNK_SIZE, width - col),
                              height=min(CHUNK_SIZE, height - row))
                
                # Read all bands at once
                data = src.read(window=window)
                
                # Check if chunk contains any data
                if np.any(data):
                    rows, cols = data[0].shape
                    
                    # Create base DataFrame with coordinates
                    row_coords, col_coords = np.meshgrid(
                        np.arange(row, row + rows),
                        np.arange(col, col + cols),
                        indexing="ij"
                    )
                    
                    chunk_df = pd.DataFrame({
                        'raster_file': file,
                        'subfolder': os.path.basename(root),
                        'x': row_coords.flatten(),
                        'y': col_coords.flatten()
                    })
                    
                    # Add each band's data
                    for band_idx, band_name in enumerate(band_names, 1):
                        chunk_df[band_name] = data[band_idx-1].flatten()
                    
                    chunk_dfs.append(chunk_df)
        
        return pd.concat(chunk_dfs, ignore_index=True) if chunk_dfs else pd.DataFrame()

# Create list of tasks
dask_dfs = [
    read_raster_in_chunks(os.path.join(root, file), file, root)
    for root, dirs, files in os.walk(raster_train_file_path)
    for file in files if file.endswith('.tif')
]

# Compute all tasks
dataframes = compute(*dask_dfs)

# Combine all DataFrames
final_df = pd.concat(dataframes, ignore_index=True)

# Debug prints
print("DataFrame shape:", final_df.shape)
print("\nDataFrame columns:", final_df.columns.tolist())
print("\nSample of data:")
print(final_df.head())

## Exploratory Data Analysis (EDA) & Feature Engineering

This code performs several operations on a DataFrame (`final_df`), converting it to a Dask DataFrame for distributed processing, renaming columns, and dropping unnecessary columns:

1. **Convert to Dask DataFrame**:
   - Converts the existing Pandas DataFrame (`final_df`) into a Dask DataFrame (`ddf`) to enable parallel, distributed processing.

2. **Rename Columns**:
   - Defines a list of new column names for Sentinel-2 bands and other data (`new_col_names`).
   - Renames `ddf` columns using this list to make them more descriptive.

3. **Drop Unneeded Columns**:
   - Drops columns such as `raster_file`, `subfolder`, `x`, `y`, and `dNBR`, keeping only essential information in the DataFrame (`df`).

4. **Display DataFrame**:
   - Uses `display()` to view the DataFrame when needed, triggering Dask's computation.

In [ ]:
# Convert to Dask DataFrame
ddf = pd.DataFrame(final_df)  # Adjust npartitions as needed

# Rename Sentinel-2 Bands columns
new_col_names = ['raster_file', 'subfolder', 'x', 'y', 'Band_1', 'Band_2', 'Band_3', 'Band_4', 'Band_5', 
                 'Band_6', 'Band_7', 'Band_8', 'Band_8A', 'Band_9', 'Band_11', 'Band_12', 'dNBR',
                 'NDVI', 'NDWI', 'Burn_Label']

# Rename columns
final_df.columns = new_col_names

# Drop columns using Dask method
df = final_df.drop(columns=['raster_file', 'subfolder', 'x', 'y', 'dNBR'])
display(df)  # Compute only when you need to display or save results

### Check Burn Class

This code checks and displays the counts of burn records in the DataFrame:

1. **Count Burn Labels**:
   - Counts occurrences in the `Burn_Label` column to determine the number of "Burn" and "Unburn" records.
   - Renames the labels: `1` to "Burn" and `0` to "Unburn" for readability.

2. **Display Counts**:
   - Prints the resulting counts to show the distribution of burn and unburned areas.

In [ ]:
# Check Burn Records
burn_counts = df['Burn_Label'].value_counts().rename(index={1: 'Burn', 0: 'Unburn'})

# Display the counts with labels
print(burn_counts)

### Downsampling

This code performs downsampling to balance the dataset by reducing the number of "Unburn" records to match the count of "Burn" records:

1. **Get Burn Count**:
   - Retrieves the count of "Burn" records from `burn_counts`.

2. **Sample Unburned Records**:
   - Selects a random sample of "Unburn" records, equal in size to the number of "Burn" records, using a fixed `random_state` for reproducibility.

3. **Combine Burn and Downsampled Unburn Records**:
   - Combines all "Burn" records with the downsampled "Unburn" sample into a new DataFrame (`downsampled_df`).

4. **Check New Burn Record Counts**:
   - Counts and displays the "Burn" and "Unburn" records in `downsampled_df` to verify balance.

In [ ]:
burn_count = burn_counts['Burn']
unburn_sample = df[df['Burn_Label'] == 0].sample(n=burn_count, random_state=42)

downsampled_df = pd.concat([df[df['Burn_Label'] == 1], unburn_sample])

# Check Burn Records
burn_counts = downsampled_df['Burn_Label'].value_counts().rename(index={1: 'Burn', 0: 'Unburn'})

# Display the counts with labels
print(burn_counts)

### Remove infinite values

This code handles the presence of infinite values and missing data in the `downsampled_df` DataFrame:

1. **Replace Infinite Values**:
   - Replaces both positive and negative infinite values (`np.inf` and `-np.inf`) with `NaN` using `replace()`. This ensures that infinite values do not interfere with further processing.

2. **Drop Rows with Missing Values**:
   - Removes any rows containing `NaN` values using `dropna()`, ensuring the DataFrame only contains valid data.

3. **Display the DataFrame**:
   - Displays the cleaned DataFrame (`downsampled_df`) for inspection.

In [ ]:
# Replacing infinite with nan 
downsampled_df.replace([np.inf, -np.inf], np.nan, inplace=True) 
  
# Dropping all the rows with nan values 
downsampled_df.dropna(inplace=True)

# Printing df 
display(downsampled_df)

### Seperate Burn_Label from DataFrame

This code separates the `Burn_Label` from the main DataFrame and ensures the label is in the correct format:

1. **Separate Burn Label**:
   - Extracts the `Burn_Label` column from `downsampled_df` into a new DataFrame (`burn_label`).

2. **Remove Burn Label from Main DataFrame**:
   - Drops the `Burn_Label` column from `downsampled_df` to ensure only the feature data remains.

3. **Convert Burn Label to Integer**:
   - Changes the data type of the `burn_label` DataFrame to `int32` to ensure consistent and efficient processing.

4. **Display Burn Label**:
   - Displays the modified `burn_label` DataFrame to verify the changes.

In [ ]:
# Seperate Burn_Label from DataFrame
burn_label = downsampled_df[['Burn_Label']]

# Drop Label from DataFrame
downsampled_df = downsampled_df.drop(columns=['Burn_Label'])

# Change type of Label to Integer Format
burn_label = burn_label.astype('int32')
display(burn_label)

### Normalization Data with MinMax Scaler

This code normalizes the feature data and saves the normalization model, while also combining it with the `Burn_Label`:

1. **List Columns**:
   - Creates a list of column names from `downsampled_df` to keep track of the column order after normalization (`cols_norm`).

2. **Normalize the Data**:
   - Imports `MinMaxScaler` from `sklearn` and fits it to the data in `downsampled_df`, which scales the values between 0 and 1.

3. **Save the Scaler**:
   - Saves the fitted `MinMaxScaler` model to a specified path (`MinMax_Scaler.pkl`) for later use.

4. **Apply Normalization**:
   - Normalizes the data by applying the `scaler` to `downsampled_df`, then converts the result back into a DataFrame (`df_norm`) with the original column names.

5. **Check Shape**:
   - Prints the shape of `df_norm` to confirm the normalization was applied correctly.

6. **Concatenate with Burn Label**:
   - Combines the normalized feature data (`df_norm`) with the `burn_label` DataFrame, aligning them by their indices and ensuring the result is a complete dataset.

7. **Display the DataFrame**:
   - Displays the final DataFrame (`df_norm`), which now includes both the normalized features and the `Burn_Label`.


In [ ]:
# Reassign the dataframe with a list of the columns
cols_norm = downsampled_df.columns.tolist()

# Import Normalize technique
scaler = MinMaxScaler()

# Normalize data
scaler.fit(downsampled_df)

# Save the scaler
scaler_save_path = r'Export Model'
save_path = os.path.join(scaler_save_path, 'MinMax_Scaler_DL.pkl')
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, 'wb') as f:
    pickle.dump(scaler, f)

# Normalize Data
df_norm = scaler.transform(downsampled_df)
df_norm = pd.DataFrame(df_norm, columns=cols_norm)

# Check df_norm shape after normalization
print("Shape of df_norm after normalization:", df_norm.shape)

# Concatenate df_norm with burn_label
df_norm = pd.concat([df_norm.reset_index(drop=True), burn_label.reset_index(drop=True)], axis=1, sort=False)
display(df_norm)

## Deep Learning Hyperparameters Analysis & Training

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check if GPU is available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    logger.info("GPU is available and will be used for training.")
    gpu_name = gpus[0].name
    logger.info(f"GPU: {gpu_name}")
else:
    logger.info("GPU is not available, training will be done on CPU.")

# Ensure df_norm is already defined and preprocessed
# Define the features and target
X = df_norm.drop(columns=['Burn_Label'])  # Features
y = df_norm['Burn_Label']  # Target

# Convert labels to one-hot encoding
y_onehot = tf.keras.utils.to_categorical(y, num_classes=y.nunique())

# Function to build the model
def build_model(hidden_layers=3):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X.shape[1],)))
    for _ in range(hidden_layers):
        model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(y_onehot.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Grid Search for Best Hyperparameters
best_params = None
best_score = 0

# Train the Model with Best Hyperparameters on Entire Dataset
model = build_model(hidden_layers=3)
history = model.fit(X, y_onehot, epochs=5, batch_size=32, verbose=1)

# Plot Accuracy and Loss Graphs
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Cross-Validation Predictions
y_pred = cross_val_predict(
    estimator=tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=lambda: build_model(hidden_layers=3),
                                                             epochs=2,
                                                             batch_size=32, verbose=1),
    X=X, y=y, cv=10
)

# Classification Report as Pandas DataFrame
report = classification_report(y, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print(report_df)

# Confusion Matrix Heatmap
cm = confusion_matrix(y, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()